# DML Analysis

In this notebook, I will examine the location of differentially methylated loci (DML) in the *C. gigas* genome. The DML were identified using `methylKit` in [this R script](https://github.com/RobertsLab/project-gigas-oa-meth/blob/master/analyses/2019-09-12-MethylKit/2019-09-12-MethylKit.Rmd).

Methods:

1. Prepare for Analyses
2. Locate Files and Set Variable Paths
3. Identify Overlaps between Genomic Feature Tracks

## 0. Prepare for Analyses

### 0a. Set Working Directory

In [1]:
pwd

'/Users/yaamini/Documents/project-gigas-oa-meth/notebooks'

In [2]:
cd ../analyses/

/Users/yaamini/Documents/project-gigas-oa-meth/analyses


In [3]:
!mkdir 2019-09-15-DML-Analysis

In [4]:
ls -F

2019-08-30-Bismark-Parameter-Testing/ 2019-09-15-DML-Analysis/
2019-09-12-MethylKit/                 README.md
2019-09-13-IGV-Verification/


In [5]:
cd 2019-09-15-DML-Analysis/

/Users/yaamini/Documents/project-gigas-oa-meth/analyses/2019-09-15-DML-Analysis


### 0b. Download Genome Feature Files

I will be using the following tracks from [this `eagle` directory](https://eagle.fish.washington.edu/trilobite/index.php?dir=Crassostrea_gigas_v9_tracks%2F):

1. Exon: Coding regions
2. Intron: Regions that are removed
3. Genes: This includes exons and introns, as well as constituent mRNA.
4. Promoters: Regions upstream of genes that could be important for regulation
5. Tranpsosable elements (_C. gigas_): Transposable elements located using information from _C. gigas_ only (see [Sam's notes](http://onsnetwork.org/kubu4/2018/08/28/transposable-element-mapping-crassostrea-virginica-genome-cvirginica_v300-using-repeatmasker-4-07/) for more information)
4. CG motifs: Regions with CGs where methylation can occur

In [6]:
!curl https://eagle.fish.washington.edu/trilobite/Crassostrea_gigas_v9_tracks/Cgigas_v9_exon.gff \
> Cgigas_v9_exon.gff

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 11.7M  100 11.7M    0     0  6888k      0  0:00:01  0:00:01 --:--:-- 6964k


In [7]:
!curl https://eagle.fish.washington.edu/trilobite/Crassostrea_gigas_v9_tracks/Cgigas_v9_intron.gff \
> Cgigas_v9_intron.gff

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 12.0M  100 12.0M    0     0  7933k      0  0:00:01  0:00:01 --:--:-- 7947k


In [8]:
!curl https://eagle.fish.washington.edu/trilobite/Crassostrea_gigas_v9_tracks/Cgigas_v9_gene.gff \
> Cgigas_v9_gene.gff

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1777k  100 1777k    0     0  4896k      0 --:--:-- --:--:-- --:--:-- 5288k


In [9]:
!curl https://eagle.fish.washington.edu/trilobite/Crassostrea_gigas_v9_tracks/Cgigas_v9_1k5p_gene_promoter.gff \
> Cgigas_v9_1k5p_gene_promoter.gff

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1848k  100 1848k    0     0  5306k      0 --:--:-- --:--:-- --:--:-- 5373k


In [20]:
!curl https://eagle.fish.washington.edu/trilobite/Crassostrea_gigas_v9_tracks/Cgigas_v9_TE.gff \
> Cgigas_v9_TE.gff

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 6325k  100 6325k    0     0  7365k      0 --:--:-- --:--:-- --:--:-- 7695k


In [10]:
!curl https://eagle.fish.washington.edu/trilobite/Crassostrea_gigas_v9_tracks/Cgigas_v9_CG.gff \
> Cgigas_v9_CG.gff

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  932M  100  932M    0     0  9789k      0  0:01:37  0:01:37 --:--:-- 9541k


In [11]:
!ls Cgigas*

Cgigas_v9_1k5p_gene_promoter.gff Cgigas_v9_gene.gff
Cgigas_v9_CG.gff                 Cgigas_v9_intron.gff
Cgigas_v9_exon.gff


## 1. Locate Relevant Files and Set Variable Path Names

### 1a. Set Variable Path Names

Setting the variable path names allows me to reuse this script with different input files or different paths to programs without manually changing the file names each time.

In [12]:
bedtoolsDirectory = "/Users/Shared/bioinformatics/bedtools2/bin/"

In [78]:
DMLlist = "../2019-09-12-MethylKit/2019-09-15-DML-Destrand-10x-Locations-100diff-NoExtras.bed"

In [14]:
exonList = "Cgigas_v9_exon.gff"

In [15]:
intronList = "Cgigas_v9_intron.gff"

In [17]:
geneList = "Cgigas_v9_gene.gff"

In [18]:
promoterList = "Cgigas_v9_1k5p_gene_promoter.gff"

In [21]:
transposableElements = "Cgigas_v9_TE.gff"

In [22]:
CGMotifList = "Cgigas_v9_CG.gff"

### 1b. Confirm Variable Path Works and Characterize Files

The BEDfiles with DML and DMR can be viewed below. Columns are are the chromosome, start position, end position, strand, and fold difference with direction. The files only have DML and DMR that were at least 50% different between the two treatments (control and elevated pCO<sub>2</sub>).

In [79]:
#Previewing the files
!head {DMLlist}

C22384	1328	1330	-100
C22628	1621	1623	100
C28982	4929	4931	100
C29914	4052	4054	-100
C29914	4052	4054	-100
C29976	649	651	-100
C30322	3482	3484	-100
C30322	3599	3601	-100
C32984	5070	5072	-100
C33708	8307	8309	100


In [80]:
#Counting the number of lines to count DML
!wc -l {DMLlist}

     628 ../2019-09-12-MethylKit/2019-09-15-DML-Destrand-10x-Locations-100diff-NoExtras.bed


In [25]:
!head {exonList}

C16582	GLEAN	CDS	35	385	.	-	0	Parent=CGI_10000001;
C17212	GLEAN	CDS	31	363	.	+	0	Parent=CGI_10000002;
C17316	GLEAN	CDS	30	257	.	+	0	Parent=CGI_10000003;
C17476	GLEAN	CDS	104	257	.	-	0	Parent=CGI_10000004;
C17476	GLEAN	CDS	34	74	.	-	2	Parent=CGI_10000004;
C17998	GLEAN	CDS	196	387	.	-	0	Parent=CGI_10000005;
C18346	GLEAN	CDS	174	551	.	+	0	Parent=CGI_10000009;
C18428	GLEAN	CDS	286	546	.	-	0	Parent=CGI_10000010;
C18964	GLEAN	CDS	203	658	.	-	0	Parent=CGI_10000011;
C18980	GLEAN	CDS	30	674	.	+	0	Parent=CGI_10000012;


In [26]:
!wc -l {exonList}

  196691 Cgigas_v9_exon.gff


In [27]:
!head {intronList}

In [28]:
!wc -l {intronList}

  176049 Cgigas_v9_intron.gff


In [29]:
!head {geneList}

C16582	GLEAN	mRNA	35	385	0.555898	-	.	ID=CGI_10000001;
C17212	GLEAN	mRNA	31	363	0.999572	+	.	ID=CGI_10000002;
C17316	GLEAN	mRNA	30	257	0.555898	+	.	ID=CGI_10000003;
C17476	GLEAN	mRNA	34	257	0.998947	-	.	ID=CGI_10000004;
C17998	GLEAN	mRNA	196	387	1	-	.	ID=CGI_10000005;
C18346	GLEAN	mRNA	174	551	1	+	.	ID=CGI_10000009;
C18428	GLEAN	mRNA	286	546	0.555898	-	.	ID=CGI_10000010;
C18964	GLEAN	mRNA	203	658	0.999572	-	.	ID=CGI_10000011;
C18980	GLEAN	mRNA	30	674	0.555898	+	.	ID=CGI_10000012;
C19100	GLEAN	mRNA	160	681	0.999955	-	.	ID=CGI_10000013;


In [30]:
!wc -l {geneList}

   28027 Cgigas_v9_gene.gff


In [31]:
!head {promoterList}

In [32]:
!wc -l {promoterList}

   28023 Cgigas_v9_1k5p_gene_promoter.gff


In [33]:
!head {transposableElements}

C21242	TRF	Tandem_Repeat	38	100	72	+	.	.
C21306	TRF	Tandem_Repeat	35	143	112	+	.	.
C21306	TRF	Tandem_Repeat	574	947	208	+	.	.
C21306	TRF	Tandem_Repeat	574	901	313	+	.	.
C21372	TRF	Tandem_Repeat	643	671	58	+	.	.
C22542	TRF	Tandem_Repeat	1727	1774	96	+	.	.
C22728	TRF	Tandem_Repeat	426	491	105	+	.	.
C23428	TRF	Tandem_Repeat	130	415	202	+	.	.
C23796	TRF	Tandem_Repeat	547	608	97	+	.	.
C24440	TRF	Tandem_Repeat	1059	1089	62	+	.	.


In [34]:
!wc -l {transposableElements}

  119786 Cgigas_v9_TE.gff


In [35]:
!head {CGMotifList}

##gff-version 3
##sequence-region scaffold360 1 280
#!Date 2013-04-23
#!Type DNA
#!Source-version EMBOSS 6.5.7.0
scaffold360	fuzznuc	nucleotide_motif	60	61	2	+	.	ID=scaffold360.1;note=*pat pattern:CG
scaffold360	fuzznuc	nucleotide_motif	96	97	2	+	.	ID=scaffold360.2;note=*pat pattern:CG
scaffold360	fuzznuc	nucleotide_motif	120	121	2	+	.	ID=scaffold360.3;note=*pat pattern:CG
scaffold360	fuzznuc	nucleotide_motif	187	188	2	+	.	ID=scaffold360.4;note=*pat pattern:CG
##gff-version 3


In [36]:
!wc -l {CGMotifList}

 10035701 Cgigas_v9_CG.gff


## 2. Identify DML Overlaps with Genomic Feature Tracks

To identify the location of DML in the *C. gigas* genome, I will use `intersect` from `bedtools`. [The BEDtools suite](http://bedtools.readthedocs.io/en/latest/content/bedtools-suite.html) allows me to easily find overlapping regions of different BEDfiles.

In [36]:
! {bedtoolsDirectory}intersectBed -h


Tool:    bedtools intersect (aka intersectBed)
Version: v2.26.0
Summary: Report overlaps between two feature files.

Usage:   bedtools intersect [OPTIONS] -a <bed/gff/vcf/bam> -b <bed/gff/vcf/bam>

	Note: -b may be followed with multiple databases and/or 
	wildcard (*) character(s). 
Options: 
	-wa	Write the original entry in A for each overlap.

	-wb	Write the original entry in B for each overlap.
		- Useful for knowing _what_ A overlaps. Restricted by -f and -r.

	-loj	Perform a "left outer join". That is, for each feature in A
		report each overlap with B.  If no overlaps are found, 
		report a NULL feature for B.

	-wo	Write the original A and B entries plus the number of base
		pairs of overlap between the two features.
		- Overlaps restricted by -f and -r.
		  Only A features with overlap are reported.

	-wao	Write the original A and B entries plus the number of base
		pairs of overlap between the two features.
		- Overlapping features restricted by -f and -r.
		  However, A fea

### 2a. CG motifs

This is more of a sanity check than anything else. If 100% of the DML do not overlap with CG motifs, we have a problem.

In [81]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {DMLlist} \
-b {CGMotifList} \
| wc -l
!echo "DML overlaps with CG motifs"

     624
DML overlaps with CG motifs


They all don't overlap, so I need to fix that after PCSGA.

### 2b. Exons

In [82]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {DMLlist} \
-b {exonList} \
| wc -l
!echo "DML overlaps with exons"

     157
DML overlaps with exons


In [83]:
! {bedtoolsDirectory}intersectBed \
-wb \
-a {DMLlist} \
-b {exonList} \
> 2019-09-15-DML-Exon.txt

In [84]:
!head 2019-09-15-DML-Exon.txt

C28982	4929	4931	100	C28982	GLEAN	CDS	4851	4993	.	-	0	Parent=CGI_10000485;
C33708	8307	8309	100	C33708	GLEAN	CDS	8220	8464	.	-	2	Parent=CGI_10001120;
scaffold100	676764	676766	100	scaffold100	GLEAN	CDS	676685	676795	.	-	0	Parent=CGI_10026448;
scaffold102	108199	108201	-100	scaffold102	GLEAN	CDS	108080	108283	.	+	0	Parent=CGI_10028421;
scaffold102	108944	108946	-100	scaffold102	GLEAN	CDS	108904	109002	.	+	0	Parent=CGI_10028421;
scaffold102	110675	110677	-100	scaffold102	GLEAN	CDS	110633	110736	.	+	2	Parent=CGI_10028421;
scaffold1024	215501	215503	100	scaffold1024	GLEAN	CDS	215426	215584	.	+	2	Parent=CGI_10028528;
scaffold1031	425965	425967	100	scaffold1031	GLEAN	CDS	425901	426024	.	+	1	Parent=CGI_10019710;
scaffold1037	361706	361708	-100	scaffold1037	GLEAN	CDS	361695	361813	.	-	1	Parent=CGI_10017467;
scaffold107	1026788	1026790	-100	scaffold107	GLEAN	CDS	1026676	1026843	.	-	1	Parent=CGI_10025081;


### 2c. Introns

In [85]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {DMLlist} \
-b {intronList} \
| wc -l
!echo "DML overlaps with introns"

     285
DML overlaps with introns


In [86]:
! {bedtoolsDirectory}intersectBed \
-wb \
-a {DMLlist} \
-b {intronList} \
> 2019-09-15-DML-Intron.txt

In [87]:
!head 2019-09-15-DML-Intron.txt

### 2d. Genes

In [88]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {DMLlist} \
-b {geneList} \
| wc -l
!echo "DML overlaps with genes"

     442
DML overlaps with genes


In [89]:
! {bedtoolsDirectory}intersectBed \
-wb \
-a {DMLlist} \
-b {geneList} \
> 2019-09-15-DML-Genes.txt

In [90]:
!head 2019-09-15-DML-Genes.txt

C28982	4929	4931	100	C28982	GLEAN	mRNA	756	6077	0.999998	-	.	ID=CGI_10000485;
C33708	8307	8309	100	C33708	GLEAN	mRNA	8220	14973	0.727981	-	.	ID=CGI_10001120;
C34158	8559	8561	100	C34158	GLEAN	mRNA	8394	13250	1	-	.	ID=CGI_10001223;
scaffold100	645217	645219	-100	scaffold100	GLEAN	mRNA	633050	652924	0.873323	+	.	ID=CGI_10026446;
scaffold100	676764	676766	100	scaffold100	GLEAN	mRNA	671285	725122	0.993892	-	.	ID=CGI_10026448;
scaffold102	107843	107845	-100	scaffold102	GLEAN	mRNA	106910	127626	0.997949	+	.	ID=CGI_10028421;
scaffold102	108199	108201	-100	scaffold102	GLEAN	mRNA	106910	127626	0.997949	+	.	ID=CGI_10028421;
scaffold102	108296	108298	-100	scaffold102	GLEAN	mRNA	106910	127626	0.997949	+	.	ID=CGI_10028421;
scaffold102	108460	108462	-100	scaffold102	GLEAN	mRNA	106910	127626	0.997949	+	.	ID=CGI_10028421;
scaffold102	108466	108468	-100	scaffold102	GLEAN	mRNA	106910	127626	0.997949	+	.	ID=CGI_10028421;


I know how many overlaps there are, but I also want to know how many unique genes have DMLs in them. For this, I will use the following code:

`cut -f9 2019-09-15-DML-Genes.txt | sort | uniq -c`

`cut` is the command that isolates the column information. Each gene has a unique end position, so I'll look at unique entries in the ninth column (`-f9`). The column is piped into `sort`, then that output is counted for unique lines by `uniq`. Finally, I'll pipe this into `wc -l` to count the number of unique genes.

In [91]:
! cut -f9 2019-09-15-DML-Genes.txt | sort | uniq -c | wc -l
!echo "unique genes overlapping with DML"

     389
unique genes overlapping with DML


### 2e. Promoters

In [92]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {DMLlist} \
-b {promoterList} \
| wc -l
!echo "DML overlaps with promoters"

      24
DML overlaps with promoters


In [93]:
! {bedtoolsDirectory}intersectBed \
-wb \
-a {DMLlist} \
-b {promoterList} \
> 2019-09-15-DML-Promoters.txt

In [94]:
!head 2019-09-15-DML-Promoters.txt

### 2f. Transposable Elements

In [95]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {DMLlist} \
-b {transposableElements} \
| wc -l
!echo "DML overlaps with transposable elements"

       8
DML overlaps with transposable elements


In [96]:
! {bedtoolsDirectory}intersectBed \
-wb \
-a {DMLlist} \
-b {transposableElements} \
> 2019-09-15-DML-TE.txt

In [97]:
!head 2019-09-15-DML-TE.txt

scaffold11	354193	354195	100	scaffold11	WUBlastX	LINE_L2	354101	354322	89	-	.	.
scaffold11	354209	354211	100	scaffold11	WUBlastX	LINE_L2	354101	354322	89	-	.	.
scaffold126	579856	579858	-100	scaffold126	WUBlastX	DNA_MuDR	579734	580051	64	+	.	.
scaffold126	579856	579858	-100	scaffold126	WUBlastX	DNA_MuDR	579743	579997	79	+	.	.
scaffold1301	958700	958702	-100	scaffold1301	WUBlastX	DNA_TcMar-Pogo	958126	959520	528	+	.	.
scaffold1599	123482	123484	-100	scaffold1599	TRF	Tandem_Repeat	123195	124139	1487	+	.	.
scaffold1631	15201	15203	-100	scaffold1631	TRF	Tandem_Repeat	15150	15205	67	+	.	.
scaffold1715	426046	426048	100	scaffold1715	WUBlastX	DNA_IS4EU	425909	426592	413	-	.	.
scaffold1715	426046	426048	100	scaffold1715	WUBlastX	DNA_IS4EU	425912	426751	348	-	.	.
scaffold1843	28003	28005	100	scaffold1843	WUBlastX	DNA_hAT-hATw	27958	28119	23	-	.	.


### 2g. No overlaps

I also want to count the number of DML that do not overlap with any features (i.e. unannotated intergenic regions). To do this, I'll use the `-v` argument in `bedtools`, which reports "those entries in A that have no overlap in B." I can specify multiple files with `-b`. I'll use exons, introns, transposable elements, and putative promoter regions.

In [98]:
! {bedtoolsDirectory}intersectBed \
-v \
-a {DMLlist} \
-b {exonList} {intronList} {transposableElements} {promoterList} \
| wc -l
!echo "DML do not overlap with exons, introns, transposable elements, or promoters"

     165
DML do not overlap with exons, introns, transposable elements, or promoters


In [99]:
! {bedtoolsDirectory}intersectBed \
-v \
-a {DMLlist} \
-b {exonList} {intronList} {transposableElements} {promoterList} \
> 2019-09-15-No-Overlap-DML.txt

In [100]:
!head 2019-09-15-No-Overlap-DML.txt

C22384	1328	1330	-100
C22628	1621	1623	100
C29914	4052	4054	-100
C29914	4052	4054	-100
C29976	649	651	-100
C30322	3482	3484	-100
C30322	3599	3601	-100
C32984	5070	5072	-100
scaffold1017	395465	395467	100
scaffold1024	1343499	1343501	100


## 3. Identify Overlaps between Other Genome Feature Tracks

### 3a. CG motifs

To fully understand my results, I also need to know where CG motifs are located with respect to the other features.

#### Exons

In [37]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {exonList} \
-b {CGMotifList} \
| wc -l
!echo "Exon overlaps with CG motifs"

  172056
Exon overlaps with CG motifs


In [38]:
! {bedtoolsDirectory}intersectBed \
-wo \
-a {exonList} \
-b {CGMotifList} \
> 2019-09-15-Exon-CGmotifs.txt

In [39]:
!head 2019-09-15-Exon-CGmotifs.txt

C16582	GLEAN	CDS	35	385	.	-	0	Parent=CGI_10000001;	C16582	fuzznuc	nucleotide_motif	41	42	2	+	.	ID=C16582.1;note=*pat pattern:CG	1
C16582	GLEAN	CDS	35	385	.	-	0	Parent=CGI_10000001;	C16582	fuzznuc	nucleotide_motif	47	48	2	+	.	ID=C16582.2;note=*pat pattern:CG	1
C16582	GLEAN	CDS	35	385	.	-	0	Parent=CGI_10000001;	C16582	fuzznuc	nucleotide_motif	67	68	2	+	.	ID=C16582.3;note=*pat pattern:CG	1
C16582	GLEAN	CDS	35	385	.	-	0	Parent=CGI_10000001;	C16582	fuzznuc	nucleotide_motif	84	85	2	+	.	ID=C16582.4;note=*pat pattern:CG	1
C16582	GLEAN	CDS	35	385	.	-	0	Parent=CGI_10000001;	C16582	fuzznuc	nucleotide_motif	93	94	2	+	.	ID=C16582.5;note=*pat pattern:CG	1
C16582	GLEAN	CDS	35	385	.	-	0	Parent=CGI_10000001;	C16582	fuzznuc	nucleotide_motif	125	126	2	+	.	ID=C16582.6;note=*pat pattern:CG	1
C16582	GLEAN	CDS	35	385	.	-	0	Parent=CGI_10000001;	C16582	fuzznuc	nucleotide_motif	177	178	2	+	.	ID=C16582.7;note=*pat pattern:CG	1
C16582	GLEAN	CDS	35	385	.	-	0	Parent=CGI_10000001;	C16582	fuzznuc	nucleotide_motif	216

#### Introns

In [40]:
!{bedtoolsDirectory}intersectBed \
-u \
-a {intronList} \
-b {CGMotifList} \
| wc -l
!echo "Intron overlaps with CG motifs"

  150884
Intron overlaps with CG motifs


In [41]:
! {bedtoolsDirectory}intersectBed \
-wo \
-a {intronList} \
-b {CGMotifList} \
> 2019-09-15-Intron-CGmotifs.txt

In [42]:
!head 2019-09-15-Intron-CGmotifs.txt

	C19392	fuzznuc	nucleotide_motif	244	245	2	+	.	ID=C19392.6;note=*pat pattern:CG	1
	C20262	fuzznuc	nucleotide_motif	690	691	2	+	.	ID=C20262.40;note=*pat pattern:CG	1
	C20262	fuzznuc	nucleotide_motif	697	698	2	+	.	ID=C20262.41;note=*pat pattern:CG	1
	C20262	fuzznuc	nucleotide_motif	739	740	2	+	.	ID=C20262.42;note=*pat pattern:CG	1
	C20262	fuzznuc	nucleotide_motif	846	847	2	+	.	ID=C20262.43;note=*pat pattern:CG	1
	C20262	fuzznuc	nucleotide_motif	867	868	2	+	.	ID=C20262.44;note=*pat pattern:CG	1
	C20334	fuzznuc	nucleotide_motif	786	787	2	+	.	ID=C20334.7;note=*pat pattern:CG	1
	C20334	fuzznuc	nucleotide_motif	815	816	2	+	.	ID=C20334.8;note=*pat pattern:CG	1
	C20412	fuzznuc	nucleotide_motif	244	245	2	+	.	ID=C20412.5;note=*pat pattern:CG	1
	C20412	fuzznuc	nucleotide_motif	286	287	2	+	.	ID=C20412.6;note=*pat pattern:CG	1


#### Genes

In [43]:
!{bedtoolsDirectory}intersectBed \
-u \
-a {geneList} \
-b {CGMotifList} \
| wc -l
!echo "gene overlaps with CG motifs"

   28015
gene overlaps with CG motifs


In [44]:
! {bedtoolsDirectory}intersectBed \
-wo \
-a {geneList} \
-b {CGMotifList} \
> 2019-09-15-Genes-CGmotifs.txt

In [45]:
!head 2019-09-15-Genes-CGmotifs.txt

C16582	GLEAN	mRNA	35	385	0.555898	-	.	ID=CGI_10000001;	C16582	fuzznuc	nucleotide_motif	41	42	2	+	.	ID=C16582.1;note=*pat pattern:CG	1
C16582	GLEAN	mRNA	35	385	0.555898	-	.	ID=CGI_10000001;	C16582	fuzznuc	nucleotide_motif	47	48	2	+	.	ID=C16582.2;note=*pat pattern:CG	1
C16582	GLEAN	mRNA	35	385	0.555898	-	.	ID=CGI_10000001;	C16582	fuzznuc	nucleotide_motif	67	68	2	+	.	ID=C16582.3;note=*pat pattern:CG	1
C16582	GLEAN	mRNA	35	385	0.555898	-	.	ID=CGI_10000001;	C16582	fuzznuc	nucleotide_motif	84	85	2	+	.	ID=C16582.4;note=*pat pattern:CG	1
C16582	GLEAN	mRNA	35	385	0.555898	-	.	ID=CGI_10000001;	C16582	fuzznuc	nucleotide_motif	93	94	2	+	.	ID=C16582.5;note=*pat pattern:CG	1
C16582	GLEAN	mRNA	35	385	0.555898	-	.	ID=CGI_10000001;	C16582	fuzznuc	nucleotide_motif	125	126	2	+	.	ID=C16582.6;note=*pat pattern:CG	1
C16582	GLEAN	mRNA	35	385	0.555898	-	.	ID=CGI_10000001;	C16582	fuzznuc	nucleotide_motif	177	178	2	+	.	ID=C16582.7;note=*pat pattern:CG	1
C16582	GLEAN	mRNA	35	385	0.555898	-	.	ID=CGI_10000001;	C16

#### Promoters

In [46]:
!{bedtoolsDirectory}intersectBed \
-u \
-a {geneList} \
-b {promoterList} \
| wc -l
!echo "promoter overlaps with CG motifs"

    5696
promoter overlaps with CG motifs


In [47]:
! {bedtoolsDirectory}intersectBed \
-wo \
-a {promoterList} \
-b {CGMotifList} \
> 2019-09-15-Promoter-CGmotifs.txt

In [48]:
!head 2019-09-15-Promoter-CGmotifs.txt

	C17476	fuzznuc	nucleotide_motif	291	292	2	+	.	ID=C17476.9;note=*pat pattern:CG	1
	C17476	fuzznuc	nucleotide_motif	389	390	2	+	.	ID=C17476.10;note=*pat pattern:CG	1
	C17476	fuzznuc	nucleotide_motif	391	392	2	+	.	ID=C17476.11;note=*pat pattern:CG	1
	C17476	fuzznuc	nucleotide_motif	398	399	2	+	.	ID=C17476.12;note=*pat pattern:CG	1
	C17476	fuzznuc	nucleotide_motif	414	415	2	+	.	ID=C17476.13;note=*pat pattern:CG	1
	C17476	fuzznuc	nucleotide_motif	439	440	2	+	.	ID=C17476.14;note=*pat pattern:CG	1
	C17476	fuzznuc	nucleotide_motif	455	456	2	+	.	ID=C17476.15;note=*pat pattern:CG	1
	C17476	fuzznuc	nucleotide_motif	460	461	2	+	.	ID=C17476.16;note=*pat pattern:CG	1
	C17476	fuzznuc	nucleotide_motif	469	470	2	+	.	ID=C17476.17;note=*pat pattern:CG	1
	C17476	fuzznuc	nucleotide_motif	488	489	2	+	.	ID=C17476.18;note=*pat pattern:CG	1


#### Transposable Elements

In [49]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {transposableElements} \
-b {CGMotifList} \
| wc -l
!echo "Transposable element overlap with CG motifs"

   82544
Transposable element overlap with CG motifs


In [50]:
! {bedtoolsDirectory}intersectBed \
-wo \
-a {transposableElements} \
-b {CGMotifList} \
> 2019-09-15-TE-CGmotifs.txt

In [51]:
!head 2019-09-15-TE-CGmotifs.txt

C21242	TRF	Tandem_Repeat	38	100	72	+	.	.	C21242	fuzznuc	nucleotide_motif	48	49	2	+	.	ID=C21242.2;note=*pat pattern:CG	1
C21242	TRF	Tandem_Repeat	38	100	72	+	.	.	C21242	fuzznuc	nucleotide_motif	57	58	2	+	.	ID=C21242.3;note=*pat pattern:CG	1
C21242	TRF	Tandem_Repeat	38	100	72	+	.	.	C21242	fuzznuc	nucleotide_motif	70	71	2	+	.	ID=C21242.4;note=*pat pattern:CG	1
C21242	TRF	Tandem_Repeat	38	100	72	+	.	.	C21242	fuzznuc	nucleotide_motif	79	80	2	+	.	ID=C21242.5;note=*pat pattern:CG	1
C21242	TRF	Tandem_Repeat	38	100	72	+	.	.	C21242	fuzznuc	nucleotide_motif	92	93	2	+	.	ID=C21242.6;note=*pat pattern:CG	1
C21306	TRF	Tandem_Repeat	574	947	208	+	.	.	C21306	fuzznuc	nucleotide_motif	664	665	2	+	.	ID=C21306.6;note=*pat pattern:CG	1
C21306	TRF	Tandem_Repeat	574	947	208	+	.	.	C21306	fuzznuc	nucleotide_motif	799	800	2	+	.	ID=C21306.7;note=*pat pattern:CG	1
C21306	TRF	Tandem_Repeat	574	947	208	+	.	.	C21306	fuzznuc	nucleotide_motif	904	905	2	+	.	ID=C21306.8;note=*pat pattern:CG	1
C21306	TRF	Tandem_Repeat	574

#### No overlaps

In [52]:
! {bedtoolsDirectory}intersectBed \
-v \
-a {CGMotifList} \
-b {exonList} {intronList} {transposableElements} {promoterList} \
| wc -l
!echo "CG motifs do not overlap with exons, introns, transposable elements, or putative promoters"

 5118363
CG motifs do not overlap with exons, introns, transposable elements, or putative promoters


In [53]:
! {bedtoolsDirectory}intersectBed \
-v \
-a {CGMotifList} \
-b {exonList} {intronList} {transposableElementsAll} {promoterList} \
> 2019-09-15-No-Overlap-CGmotifs.txt

/bin/sh: {bedtoolsDirectory}intersectBed: command not found


In [54]:
!head 2019-09-15-No-Overlap-CGmotifs.txt

### 3b. Transposable Elements

It's also good to know where transposable elements overlap with the other feature tracks.

#### Exons

In [55]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {exonList} \
-b {transposableElements} \
| wc -l
!echo "Exon overlaps with transposable elements"

    2597
Exon overlaps with transposable elements


In [56]:
! {bedtoolsDirectory}intersectBed \
-wb \
-a {exonList} \
-b {transposableElements} \
> 2019-09-15-Exon-TE.txt

In [57]:
!head 2019-09-15-Exon-TE.txt

C22430	GLEAN	CDS	263	493	.	+	0	Parent=CGI_10000081;	C22430	WUBlastX	DNA_TcMar-Tc2	230	493	60	+	.	.
C23316	GLEAN	CDS	929	1111	.	-	0	Parent=CGI_10000116;	C23316	WUBlastX	LTR_Ngaro	929	1675	117	-	.	.
C23316	GLEAN	CDS	953	1111	.	-	0	Parent=CGI_10000116;	C23316	WUBlastX	LTR_Ngaro	953	1489	157	-	.	.
C23316	GLEAN	CDS	644	660	.	-	0	Parent=CGI_10000116;	C23316	WUBlastX	LTR_Ngaro	547	660	32	-	.	.
C23734	GLEAN	CDS	512	712	.	-	0	Parent=CGI_10000149;	C23734	WUBlastX	LTR_Gypsy	23	724	246	-	.	.
C23734	GLEAN	CDS	512	712	.	-	0	Parent=CGI_10000149;	C23734	WUBlastX	LTR_Gypsy	26	964	217	-	.	.
C24320	GLEAN	CDS	1545	1904	.	-	0	Parent=CGI_10000190;	C24320	WUBlastX	LTR_ERV1	1545	1904	27	-	.	.
C24604	GLEAN	CDS	792	989	.	-	2	Parent=CGI_10000200;	C24604	WUBlastX	DNA_MuDR	792	989	42	-	.	.
C25496	GLEAN	CDS	2006	2074	.	+	0	Parent=CGI_10000243;	C25496	WUBlastX	LINE_CR1	2006	2536	27	+	.	.
C26320	GLEAN	CDS	2740	2964	.	+	0	Parent=CGI_10000297;	C26320	WUBlastX	LINE_L2	2710	2964	75	+	.	.


#### Introns

In [58]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {intronList} \
-b {transposableElements} \
| wc -l
!echo "Intron overlaps with transposable elements"

   18989
Intron overlaps with transposable elements


In [59]:
! {bedtoolsDirectory}intersectBed \
-wb \
-a {intronList} \
-b {transposableElements} \
> 2019-09-15-Intron-TE.txt

In [60]:
!head 2019-09-15-Intron-TE.txt

	C26674	TRF	Tandem_Repeat	2430	2483	108	+	.	.
	C26674	TRF	Tandem_Repeat	2860	2913	74	+	.	.
	C26936	WUBlastX	LTR_Gypsy	1323	1346	20	-	.	.
	C26936	WUBlastX	LTR_Gypsy	2202	3359	178	-	.	.
	C26936	WUBlastX	LTR_Gypsy	2205	2915	412	-	.	.
	C26936	WUBlastX	LTR_Gypsy	1613	2275	350	-	.	.
	C26936	WUBlastX	LTR_Gypsy	2271	3059	408	-	.	.
	C26936	WUBlastX	LTR_Gypsy	1484	2260	314	-	.	.
	C26936	WUBlastX	LTR_Gypsy-Gmr1	2271	2831	413	-	.	.
	C28348	WUBlastX	LTR_Gypsy	3411	3479	23	+	.	.


#### Genes

In [61]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {geneList} \
-b {transposableElements} \
| wc -l
!echo "gene overlaps with transposable elements"

   11748
gene overlaps with transposable elements


In [62]:
! {bedtoolsDirectory}intersectBed \
-wb \
-a {geneList} \
-b {transposableElements} \
> 2019-09-15-Gene-TE.txt

In [63]:
!head 2019-09-15-Gene-TE.txt

C22430	GLEAN	mRNA	263	493	0.555898	+	.	ID=CGI_10000081;	C22430	WUBlastX	DNA_TcMar-Tc2	230	493	60	+	.	.
C23316	GLEAN	mRNA	929	1111	0.999999	-	.	ID=CGI_10000116;	C23316	WUBlastX	LTR_Ngaro	929	1675	117	-	.	.
C23316	GLEAN	mRNA	953	1111	0.999999	-	.	ID=CGI_10000116;	C23316	WUBlastX	LTR_Ngaro	953	1489	157	-	.	.
C23316	GLEAN	mRNA	644	660	0.999999	-	.	ID=CGI_10000116;	C23316	WUBlastX	LTR_Ngaro	547	660	32	-	.	.
C23734	GLEAN	mRNA	512	712	1	-	.	ID=CGI_10000149;	C23734	WUBlastX	LTR_Gypsy	23	724	246	-	.	.
C23734	GLEAN	mRNA	512	712	1	-	.	ID=CGI_10000149;	C23734	WUBlastX	LTR_Gypsy	26	964	217	-	.	.
C24320	GLEAN	mRNA	1545	1904	0.999372	-	.	ID=CGI_10000190;	C24320	WUBlastX	LTR_ERV1	1545	1904	27	-	.	.
C24604	GLEAN	mRNA	792	989	0.997918	-	.	ID=CGI_10000200;	C24604	WUBlastX	DNA_MuDR	792	989	42	-	.	.
C25496	GLEAN	mRNA	2006	2074	0.998748	+	.	ID=CGI_10000243;	C25496	WUBlastX	LINE_CR1	2006	2536	27	+	.	.
C26320	GLEAN	mRNA	2740	2964	0.481188	+	.	ID=CGI_10000297;	C26320	WUBlastX	LINE_L2	2710	2964	75	+	.	.


#### Promoters

In [64]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {promoterList} \
-b {transposableElements} \
| wc -l
!echo "promoter overlaps with transposable elements"

    3966
promoter overlaps with transposable elements


In [65]:
! {bedtoolsDirectory}intersectBed \
-wb \
-a {promoterList} \
-b {transposableElements} \
> 2019-09-15-Promoter-TE.txt

In [66]:
!head 2019-09-15-Promoter-TE.txt

	C22430	WUBlastX	DNA_TcMar-Tc2	230	493	60	+	.	.
	C23316	WUBlastX	LTR_Ngaro	929	1675	117	-	.	.
	C23316	WUBlastX	LTR_Ngaro	953	1489	157	-	.	.
	C23734	WUBlastX	LTR_Gypsy	23	724	246	-	.	.
	C23734	WUBlastX	LTR_Gypsy	26	964	217	-	.	.
	C24608	WUBlastX	DNA_En-Spm	1276	2823	1086	-	.	.
	C25496	WUBlastX	LINE_L1	1457	1753	124	+	.	.
	C25496	WUBlastX	LINE_CR1	353	1033	105	+	.	.
	C26320	WUBlastX	LINE_L2	2710	2964	75	+	.	.
	C26472	WUBlastX	LTR_Pao	173	730	39	-	.	.
